In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import shutil
import hashlib
from collections import OrderedDict
import pandas as pd
import time
import pickle
from utils import datasets

In [3]:
# Try on small subset ..


HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
frames_dir = '/media/dean/datastore/datasets/kache_ai/frames_dev/'
#bag_dirs = os.listdir(frames_dir)
save_dir = os.path.join(frames_dir,'kache_set/')
os.makedirs(save_dir, 0o755 , exist_ok = True )

img_data = []
lookup_table = {}

#for bag in [frames_dir]:
#if frames_dir != save_dir or '.csv' not in frames_dir:
img_dict = {}
bag_path = os.path.join(frames_dir)
img_dict['bag_path'] = frames_dir
img_dict['frames'] = []
img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

# Get all images
imgs = glob.glob(os.path.join(bag_path,'**/', '*.jpg'))
#imgs = glob.glob(os.path.join(bag_path, '*.jpg'))
print(imgs)

for img in set(imgs):
    print(img)
    d = {}
    d['frame_path'] = img
    hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
    hash_object = hashlib.sha1(str.encode(hex_val))
    hex_dig = hash_object.hexdigest()
    shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+'.jpg'))
    d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
    d['hash_key'] = hex_dig
    d['hash_val'] = hex_val
    lookup_table[d['hash_key']] = d['hash_val'] 
    old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

    for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
        d['bag_name'] = log[0]
        d['time_sec'] = log[1]
        d['time_readable'] = time.ctime(int(log[1]))

        d['time_nsec'] = log[2]
        d['GPS'] = log[3]
        d['v-ego'] = log[4]

        d['key_event'] = log[5]
        d['frame'] = log[6]

    if d.get('frame', None): img_dict['frames'].append(d)
if img_dict['frames']: img_data.append((bag_path,img_dict))

pickle_file = os.path.join(save_dir, 'kacheai_logs.pickle')
pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
print('Saving to Pickle File:', pickle_file)
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)



['/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0108.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0064.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0204.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0221.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0045.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0113.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0206.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0239.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0074.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0248.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0065.jpg', '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-fra

/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py:41: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0019.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0127.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0101.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0242.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0096.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0203.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0071.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0131.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0145.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0141.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0099.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0246.jpg
/media/dean/datastore/da

/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0193.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0234.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0122.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0178.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0203.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0200.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0088.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0217.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0205.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0140.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0138.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0097.jpg
/media/dean/datastore/datasets/kache_ai/frame

/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0022.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0193.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0054.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0133.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0079.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0072.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0173.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0219.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0227.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0135.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0056.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0082.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame023

/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0184.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0230.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0074.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0232.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frame0132.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0048.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0083.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0179.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0192.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0041.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frames-frames-frame0029.jpg
/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0022.jpg
/media/dean/datastore/datasets/kache_a

In [4]:
print(img_data)

[('/media/dean/datastore/datasets/kache_ai/frames_dev/', {'bag_path': '/media/dean/datastore/datasets/kache_ai/frames_dev/', 'frames': [{'frame_path': '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0234.jpg', 'dataset_path': '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/e81e315a0b7620dffb3fc302a65fe5b28f8a4b1d.jpg', 'hash_key': 'e81e315a0b7620dffb3fc302a65fe5b28f8a4b1d', 'hash_val': '^^media^^dean^^datastore^^datasets^^kache_ai^^frames_dev^^frames^^frame0234.jpg---^^media^^dean^^datastore^^datasets^^kache_ai^^frames_dev^^frames.csv', 'bag_name': 'bagfile_18-10-05_0914_5.bag', 'time_sec': 1538759724, 'time_readable': 'Fri Oct  5 10:15:24 2018', 'time_nsec': 552900568, 'GPS': '$GNGGA,171525.70,3744.90769,N,12224.23711,W,1,12,0.65,13.6,M,-29.8,M,,*4C\r', 'v-ego': 9.425000190734863, 'key_event': 'c', 'frame': 'frame0234.jpg'}, {'frame_path': '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0187.jpg', 'dataset_path': '/media/dean/datastore/dat

In [5]:


WORKING_DIR = '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/'
kache_set = datasets.DataFormatter(annotations_list = os.path.join(WORKING_DIR,'kacheai_logs.pickle'), image_list = WORKING_DIR, input_format = datasets.Format.kache,
                                 output_path = os.path.join(WORKING_DIR,'scalabel_files/'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False)

Initializing Annotation Pipeline via Darknet ... \o/ \o/
IMAGE_ANN: [{'category': 'car', 'box2d': {'x1': 611.553150177002, 'y1': 564.0104789733887, 'x2': 689.6530265808105, 'y2': 638.2831001281738}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 422.6905517578125, 'y1': 571.9694137573242, 'x2': 519.2034301757812, 'y2': 639.2267532348633}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 875.3439331054688, 'y1': 492.4469299316406, 'x2': 1178.2503051757812, 'y2': 755.5492858886719}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 512.6547107696533, 'y1': 568.9809055328369, 'x2': 539.5165538787842, 'y2': 594.6936550140381}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 580.7574119567871, 'y1': 567.028356552124, 'x2': 624.8628273010254, 'y2': 606.3955936431885}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 218.819606781005

LABEL: {'id': 996, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None, 'box3d': None, 'box2d': {'x1': 708.5639247894287, 'y1': 577.9111652374268, 'x2': 727.5039463043213, 'y2': 595.5811443328857}, 'category': 'car'}
LABEL: {'id': 997, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None, 'box3d': None, 'box2d': {'x1': 105.63735389709473, 'y1': 581.2513246536255, 'x2': 160.28171348571777, 'y2': 601.672381401062}, 'category': 'car'}
IMAGE_ANN: [{'category': 'car', 'box2d': {'x1': 619.9741592407227, 'y1': 569.6876220703125, 'x2': 750.8438339233398, 'y2': 665.72607421875}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 196.24215698242188, 'y1': 501.99410247802734, 'x2': 518.7246398925781, 'y2': 734.1584854125977}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 837.2068099975586, 'y1': 570.29

IMAGE_ANN: [{'category': 'car', 'box2d': {'x1': 601.6006889343262, 'y1': 550.1125946044922, 'x2': 716.5818061828613, 'y2': 647.0070343017578}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 530.7385406494141, 'y1': 574.2850456237793, 'x2': 587.9494476318359, 'y2': 619.7828254699707}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 793.8704071044922, 'y1': 471.42991638183594, 'x2': 1245.6654815673828, 'y2': 800.5542144775391}, 'manualAttributes': False, 'manual': False}, {'category': 'traffic sign', 'box2d': {'x1': 440.3413028717041, 'y1': 561.246826171875, 'x2': 462.5659236907959, 'y2': 580.2738037109375}, 'manualAttributes': False, 'manual': False}]
LABEL: {'id': 1932, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None, 'box3d': None, 'box2d': {'x1': 601.6006889343262, 'y1': 550.1125946044922, 'x2': 716.5818061828613, 'y2': 647.0070343017578}, 'categor

LABEL: {'id': 3034, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None, 'box3d': None, 'box2d': {'x1': 63.09501647949219, 'y1': 567.0777034759521, 'x2': 152.44700622558594, 'y2': 599.4198551177979}, 'category': 'car'}
LABEL: {'id': 3035, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None, 'box3d': None, 'box2d': {'x1': 596.4001770019531, 'y1': 581.6758060455322, 'x2': 618.1248474121094, 'y2': 602.0970211029053}, 'category': 'car'}
LABEL: {'id': 3036, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None, 'box3d': None, 'box2d': {'x1': 593.6946296691895, 'y1': 566.106972694397, 'x2': 613.5966300964355, 'y2': 582.858603477478}, 'category': 'traffic sign'}
LABEL: {'id': 3037, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': Non

IMAGE_ANN: [{'category': 'car', 'box2d': {'x1': 322.1149673461914, 'y1': 515.1976318359375, 'x2': 512.1323471069336, 'y2': 674.28125}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 778.2170715332031, 'y1': 565.9653472900391, 'x2': 971.1213073730469, 'y2': 699.0258636474609}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 526.1458053588867, 'y1': 582.9823570251465, 'x2': 581.8380813598633, 'y2': 628.2840003967285}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 18.78302001953125, 'y1': 536.9671020507812, 'x2': 291.7530517578125, 'y2': 898.9658813476562}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 638.2187118530273, 'y1': 576.4084243774414, 'x2': 716.2771377563477, 'y2': 641.5023422241211}, 'manualAttributes': False, 'manual': False}, {'category': 'car', 'box2d': {'x1': 761.7355499267578, 'y1': 581.5471000671387, 'x2': 808.1483612060547, 'y2': 631.2

LABEL: {'id': 5088, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None, 'box3d': None, 'box2d': {'x1': 718.368574142456, 'y1': 579.7817153930664, 'x2': 737.740800857544, 'y2': 605.1832504272461}, 'category': 'car'}
LABEL: {'id': 5089, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None, 'box3d': None, 'box2d': {'x1': 482.6872673034668, 'y1': 545.1950378417969, 'x2': 576.7226448059082, 'y2': 640.1470031738281}, 'category': 'car'}
LABEL: {'id': 5090, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None, 'box3d': None, 'box2d': {'x1': 743.2436084747314, 'y1': 560.6933288574219, 'x2': 763.624189376831, 'y2': 580.0121154785156}, 'category': 'traffic sign'}
LABEL: {'id': 5091, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'manual': False, 'poly2d': None,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
kache_set.export(format = datasets.Format.scalabel, paginate = False)

In [7]:
# 86469: 4.864921, 4.839282 avg loss, 0.000080 rate, 13.939101 seconds, 177088512 images


In [8]:
# Try on small subset ..


HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
frames_dir = '/media/dean/datastore/datasets/kache_ai/frames/'
bag_dirs = os.listdir(frames_dir)
save_dir = os.path.join(frames_dir,'kache_set/')
os.makedirs(save_dir, 0o755 , exist_ok = True )

img_data = []
lookup_table = {}

for bag in bag_dirs:
#if frames_dir != save_dir or '.csv' not in frames_dir:
img_dict = {}
bag_path = os.path.join(frames_dir)
img_dict['bag_path'] = frames_dir
img_dict['frames'] = []
img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

# Get all images
imgs = glob.glob(os.path.join(bag_path,'**/', '*.jpg'))
#imgs = glob.glob(os.path.join(bag_path, '*.jpg'))
print(imgs)

for img in set(imgs):
    print(img)
    d = {}
    d['frame_path'] = img
    hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
    hash_object = hashlib.sha1(str.encode(hex_val))
    hex_dig = hash_object.hexdigest()
    shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+'.jpg'))
    d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
    d['hash_key'] = hex_dig
    d['hash_val'] = hex_val
    lookup_table[d['hash_key']] = d['hash_val'] 
    old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

    for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
        d['bag_name'] = log[0]
        d['time_sec'] = log[1]
        d['time_readable'] = time.ctime(int(log[1]))

        d['time_nsec'] = log[2]
        d['GPS'] = log[3]
        d['v-ego'] = log[4]

        d['key_event'] = log[5]
        d['frame'] = log[6]

    if d.get('frame', None): img_dict['frames'].append(d)
if img_dict['frames']: img_data.append((bag_path,img_dict))

pickle_file = os.path.join(save_dir, 'kacheai_logs.pickle')
pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
print('Saving to Pickle File:', pickle_file)
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)



In [9]:


WORKING_DIR = '/media/dean/datastore/datasets/kache_ai'
kache_set = datasets.DataFormatter(annotations_list = '/media/dean/datastore/datasets/kache_ai/frames/kacheai_logs.pickle', image_list = '/media/dean/datastore/datasets/kache_ai/frames/', input_format = datasets.Format.kache,
                                 output_path = os.path.join(WORKING_DIR,'scalabel_files'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False)

In [10]:
kache_set.export(format = datasets.Format.scalabel, paginate = True)